In [1]:
import torch
import torchvision

print("torch:", torch.__version__)
print("torchvision:", torchvision.__version__)
print("CUDA available:", torch.cuda.is_available())


torch: 2.9.0+cu126
torchvision: 0.24.0+cu126
CUDA available: True


In [4]:
!pip install -q anomalib timm lightning


In [27]:
import torch

from anomalib.models import Patchcore
from anomalib.data.datamodules.image.mvtecad import MVTecAD
from anomalib.engine import Engine

print("CUDA available:", torch.cuda.is_available())



CUDA available: True


In [9]:
datamodule = MVTecAD(
    root="./mvtec",
    category="bottle",        # change category here
    train_batch_size=16,
    eval_batch_size=16,
    num_workers=2,
)


In [7]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [8]:
!kaggle datasets download -d ipythonx/mvtec-ad


Dataset URL: https://www.kaggle.com/datasets/ipythonx/mvtec-ad
License(s): copyright-authors
 98% 4.82G/4.91G [00:16<00:00, 723MB/s]
100% 4.91G/4.91G [00:16<00:00, 322MB/s]


In [9]:
!unzip -q mvtec-ad.zip -d ./mvtec


In [19]:
!ls ./mvtec


bottle	 carpet    leather	pill	    tile	wood
cable	 grid	   license.txt	readme.txt  toothbrush	zipper
capsule  hazelnut  metal_nut	screw	    transistor


In [20]:
from anomalib.data.datamodules.image.mvtecad import MVTecAD

datamodule = MVTecAD(
    root="./mvtec",
    category="bottle",     # you can change this
    train_batch_size=16,
    eval_batch_size=16,
    num_workers=2,
)


In [21]:
from anomalib.models import Ganomaly
model = Ganomaly()


In [22]:
from anomalib.engine import Engine

engine = Engine(
    max_epochs=20,
    accelerator="gpu",
    devices=1,
)


In [23]:
engine.fit(model=model, datamodule=datamodule)


INFO:lightning_fabric.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:lightning_fabric.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name               ┃ Type              ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ pre_processor      │ PreProcessor      │      0 │ train │     0 │
│ 1 │ post_processor     │ PostProcessor     │      0 │ train │     0 │
│ 2 │ evaluator          │ Evaluator         │      0 │ train │     0 │
│ 3 │ model              │ GanomalyModel     │  188 M │ train │     0 │
│ 4 │ generator_loss     │ GeneratorLoss     │      0 │ train │     0 │
│ 5 │ discriminator_loss │ DiscriminatorLoss │      0 │ train │     0 │
└───┴────────────────────┴───────────────────┴────────┴───────┴───────┘

Trainable params: 188 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 188 M                                                                                                
Total estimated model params size (MB): 754                                                                        
Modules in train mode: 112                                                                                         
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

/usr/local/lib/python3.12/dist-packages/lightning/pytorch/core/module.py:522: You called 
`self.log('generator_loss', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`

/usr/local/lib/python3.12/dist-packages/lightning/pytorch/utilities/data.py:79: Trying to infer the `batch_size` 
from an ambiguous collection. The batch size we found is 3. To avoid any miscalculations, use `self.log(..., 
batch_size=batch_size)`.

/usr/local/lib/python3.12/dist-packages/lightning/pytorch/core/module.py:522: You called 
`self.log('discriminator_loss', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`

INFO:lightning_fabric.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [28]:
model = Patchcore(
    backbone="resnet50",   # best default
)


model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

In [ ]:
datamodule = MVTecAD(
    root="./mvtec",
    category="bottle",       
    train_batch_size=16,
    eval_batch_size=4,       
    num_workers=2,
)


In [ ]:
model = Patchcore(
    backbone="resnet18",         
    coreset_sampling_ratio=0.005 
)


In [ ]:
engine = Engine(
    accelerator="gpu",
    devices=1,
    default_root_dir="./anomalib_results",  
    enable_progress_bar=False,              
)


In [ ]:
engine.fit(
    model=model,
    datamodule=datamodule,
)


In [ ]:
# ===== ONE-CELL PATCHCORE (NO DOWNLOAD, NO KAGGLE, NO HF) =====

# 1. Install anomalib
!pip install -q anomalib timm lightning

# 2. Imports
import torch
from anomalib.models import Patchcore
from anomalib.data.datamodules.image.mvtecad import MVTecAD
from anomalib.engine import Engine

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

# 3. Create DataModule
datamodule = MVTecAD(
    root="/content/mvtec",   # <-- CHANGE IF YOUR PATH IS DIFFERENT
    category="bottle",
    train_batch_size=16,
    eval_batch_size=4,
    num_workers=2,
)

# 🔥 CRITICAL FIX: disable anomalib auto-download
datamodule.prepare_data = lambda: None

# 4. PatchCore (memory-safe)
model = Patchcore(
    backbone="resnet18",
    coreset_sampling_ratio=0.005,
)

# 5. Engine
engine = Engine(
    accelerator="gpu",
    devices=1,
    default_root_dir="./anomalib_results",
    enable_progress_bar=False,
)

# 6. Run PatchCore
engine.fit(model=model, datamodule=datamodule)
engine.test(model=model, datamodule=datamodule)

print("✅ PatchCore finished successfully")
